# 문제 6

[Kaggle 형] train_prob.csv로 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (default, Oct 17 2019, 06:10:02) 
[GCC 8.3.0]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [3]:
df_train = pd.read_csv('train_prob.csv', index_col = 'id')
df_test = pd.read_csv('test_prob.csv', index_col = 'id')
s_kaggle_ans = pd.read_csv('test_prob_ans.csv', index_col = 'id')['failure']

In [6]:
df_train = df_train.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)
df_test = df_test.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna()
)

In [7]:
(
    df_train['product_code'].value_counts(),
    df_test['product_code'].value_counts(),
)

(C    5765
 E    5343
 B    5250
 A    5100
 Name: product_code, dtype: int64,
 D    5112
 Name: product_code, dtype: int64)

In [10]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
imp = IterativeImputer(
    estimator = LinearRegression(fit_intercept = True),
    random_state=123
)
df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index = x.index, columns = X_imp)
)
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index = x.index, columns = X_imp)
)
X_mean  = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))
df_test[X_mean] = df_test.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))

In [12]:
m = pd.concat([df_train['loading'], df_test['loading']]).mean()
df_train['loading'] = df_train['loading'].fillna(m)
df_test['loading'] = df_test['loading'].fillna(m)
df_train['loading_log'] = np.log(df_train['loading'])
df_test['loading_log'] = np.log(df_test['loading'])

- 문제 1.

na_1 = measurement_3의 결측여부

na_2 = measurement_3의 결측여부

failure 연관성이 있다.

- 문제 2.

loading_log = np.log(log)

attribute_0와 attribute_1은 failure와 상관없음

loading과 product_code는 상관없음

- 문제 3.

loading의 결측치: loading의 평균으로 처리 

std: loading, measurement_0~17  pt: na_1, na_2 + LR = AUC: 0.5792951262053387

SFS: ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17', 'na_1'] + LR = AUC: 0.5838326230092876
        
- 문제 4.

LDA: Transformer ['measurement_0~17']
     Predictor ?
        
PCA n_components = 7 : std ['measurement_0~17'] + LR : 0.581757510516433
        
- 문제 5

GridSearch + RandomForest: {'n_estimators': 15, 'max_depth': 7, 'min_samples_split': 512},: 0.5687712018291998

# Kaggle형 풀이 단계

Step 0: Kaggle용 데이터셋을 만든다.

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다

Step 3: 모델 선택 루틴을 만듭니다.

|id|failure|
|----|----|
|16115| 0.1|
|16116| 0.2|

....	

Step 4: 모델 개선 작업을 합니다.

## Step1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

In [14]:
X_all = df_test.columns.tolist()
np.array(X_all)

array(['product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12',
       'measurement_13', 'measurement_14', 'measurement_15',
       'measurement_16', 'measurement_17', 'na_1', 'na_2', 'loading_log'],
      dtype='<U14')

In [13]:
df_train['product_code'].unique(), df_test['product_code'].unique()

(array(['A', 'B', 'C', 'E'], dtype=object), array(['D'], dtype=object))

In [16]:
from sklearn.model_selection import GroupKFold
gkf = GroupKFold(4)
for train_idx, valid_idx in gkf.split(df_train[X_all], df_train['failure'], groups = df_train['product_code']):
    print(
        df_train.iloc[train_idx]['product_code'].unique(), 
        df_train.iloc[valid_idx]['product_code'].unique()
    )

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


In [31]:
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.metrics import roc_auc_score

gkf = GroupKFold(4)
hist = list()
def eval_model(model_name, clf):
    """
        모델의 인스턴스를 받아 검증 결과(AUC) 를 구합니다.
        1. GroupKFold 분리 검증 - df_train, groups = product_code
        2. df_train 대한 교차검증결과에 대한 AUC를 구합니다.
        3. 주어진 모델명으로 평가 결과를 저장합니다. Format: Valid: {:.5f}±{:.5f}, Train: {:.5f}±{:.5f}
        4. 가장 최근의 수행결과를 보여 주어 선택하는데 활용하도록 합니다.
    Parameters:
        model_name: str, 모델의 이름,
        clf: sklearn object, 모델 인스턴스
    """
    result = cross_validate(
        clf, df_train[X_all], df_train['failure'], groups = df_train['product_code'], scoring = 'roc_auc', cv = gkf, 
        return_train_score = True
    )
    result_str = "Valid: {:.5f}±{:.5f}, Train: {:.5f}±{:.5f}".format(
        result['test_score'].mean(), result['test_score'].std(),
        result['train_score'].mean(), result['train_score'].std()
    )
    hist.append(
        pd.Series(['model_name', result_str], index = ['name', 'result'])
    )
    display(
        pd.DataFrame(hist).groupby('name').last()
    )
    return result_str

# 모델 선택 루틴입니다. (Step 3)
def select_model(clf):
    """
        1. 전체 학습데이터(df_train)로 학습을 합니다. 
        2. df_test에 대한 예측을 합니다.
        3. 예측 결과를 출력양식(id, failure)에 맞춰 csv파일을 만듭니다.
        4. 자가 채점을 위한 예측 결과를 반환합니다.
    Parameters: 
        clf: sklearn object, 모델 인스턴스
    Returns: 1차원 np.ndarray
        예측 결과
    """
    clf.fit(df_train[X_all], df_train['failure'])
    prd = clf.predict_proba(df_test[X_all])[:, 1]
    pd.DataFrame({
        'id': df_test.index,
        'failure': prd
    }).to_csv('answer6.csv', index = None)
    return prd

## Step2: Baseline 모델을 만듭니다.

std: \['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17'\] pt: \[ 'na_1'\] -> LR

In [25]:
from sklearn.linear_model import LogisticRegression

ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])

clf_lr = make_pipeline(ct, LogisticRegression(solver='lbfgs'))
eval_model('baseline', clf_lr)

,result
name,
model_name,"Valid: 0.58937±0.00380, Train: 0.59190±0.00146"


'Valid: 0.58937±0.00380, Train: 0.59190±0.00146'

## Step3: 모델 선택 루틴을 만듭니다.

id,failure

16115, 0.1

16116, 0.2

In [32]:
prd = select_model(clf_lr)
print("자가채점:", roc_auc_score(s_kaggle_ans, prd))

자가채점: 0.5883911870503598


Int64Index([16115, 16116, 16117, 16118, 16119, 16120, 16121, 16122, 16123,
            16124,
            ...
            21217, 21218, 21219, 21220, 21221, 21222, 21223, 21224, 21225,
            21226],
           dtype='int64', name='id', length=5112)